In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame()
df = pd.read_csv("train_proper_csv_attack_types.csv",header = None)
df = df.rename(columns=df.iloc[0])
df = df.drop(df.index[0])
df = df.drop('unknown2',axis=1)

C:\Users\Rupesh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df1 = pd.DataFrame()
df1 = pd.read_csv("test_proper_csv_attack_types.csv",header = None)
df1 = df1.rename(columns=df1.iloc[0])
df1 = df1.drop(df1.index[0])
df1 = df1.drop('unknown2',axis=1)

C:\Users\Rupesh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_tot = pd.concat([df,df1])

In [5]:
def bool_attack(x):
    if(x != "normal"):
        return "attack"
    else:
        return "normal"

In [6]:
from sklearn.feature_selection import SelectFpr
from sklearn.feature_selection import chi2
from sklearn.preprocessing import Normalizer

df_tot_1 = df_tot.drop('unknown1',axis=1)
df_output = df_tot['unknown1'].apply(bool_attack)
df_tot_dummies = pd.get_dummies(df_tot_1,columns = ["protocol_type","service","flag"])
x_normalise = Normalizer().fit(df_tot_dummies)
df_tot_dummies = x_normalise.transform(df_tot_dummies)
x_new = SelectFpr(chi2, alpha=0.05).fit_transform(df_tot_dummies, df_output)

In [7]:
import pickle

random_forest_clf = pickle.load(open('random_forest_test.sav', 'rb'))
AdaBoost_clf = pickle.load(open('AdaBoost_test.sav', 'rb'))
svc_clf = pickle.load(open('svc_test.sav', 'rb'))

In [56]:
X_train = x_new[:125000]
y_train = df_output.iloc[:125000]
X_test = x_new[125000:]
y_test = df_output.iloc[125000:]

In [9]:
from sklearn.metrics import classification_report

In [57]:
def convert_bool(x):
    if(x == "attack"):
        return 1
    else:
        return 0
    
y_train = y_train.apply(convert_bool)

In [58]:
rf_pred = random_forest_clf.predict(X_train)
rf_conf = random_forest_clf.predict_proba(X_train)
ada_pred = AdaBoost_clf.predict(X_train)
ada_conf = AdaBoost_clf.predict_proba(X_train)
svc_pred = svc_clf.predict(X_train)
svc_conf = svc_clf.predict_proba(X_train)

In [59]:
conf_vec_list=[]
clf_vec_list=[]
state_vec_list=[]

In [60]:
for i in range(len(rf_pred)):
    tmp1=[]
    tmp1.append(convert_bool(rf_pred[i]))
    tmp1.append(convert_bool(ada_pred[i]))
    tmp1.append(convert_bool(svc_pred[i]))
    clf_vec_list.append(tmp1)
    state_vec_list.append(tmp1+list(x_new[i]))
    
    tmp1=[]
    tmp1.append(rf_conf[i])
    tmp1.append(ada_conf[i])
    tmp1.append(svc_conf[i])
    conf_vec_list.append(tmp1)

In [20]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
# fix random seed for reproducibility
np.random.seed(7)

In [48]:
len(x_new[0])

36

In [61]:
# create model
model = Sequential()
model.add(Dense(50, input_dim=39, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [62]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [63]:
# Fit the model
model.fit(np.array(state_vec_list), y_train, epochs=50, batch_size=10)

Epoch 1/50
125000/125000 [==============================] - 57s - loss: 0.0046 - acc: 0.9994    
Epoch 2/50
125000/125000 [==============================] - 34s - loss: 0.0016 - acc: 0.9998    
Epoch 3/50
125000/125000 [==============================] - 29s - loss: 0.0016 - acc: 0.9998    
Epoch 4/50
125000/125000 [==============================] - 31s - loss: 0.0016 - acc: 0.9998    
Epoch 5/50
125000/125000 [==============================] - 28s - loss: 0.0016 - acc: 0.9998    
Epoch 6/50
125000/125000 [==============================] - 33s - loss: 0.0015 - acc: 0.9998    
Epoch 7/50
125000/125000 [==============================] - 38s - loss: 0.0014 - acc: 0.9998    
Epoch 8/50
125000/125000 [==============================] - 34s - loss: 0.0015 - acc: 0.9998    
Epoch 9/50
125000/125000 [==============================] - 23s - loss: 0.0014 - acc: 0.9998    
Epoch 10/50
125000/125000 [==============================] - 28s - loss: 0.0014 - acc: 0.9998    
Epoch 11/50
125000/125000 [==

In [64]:
rf_pred_ts = random_forest_clf.predict(X_test)
ada_pred_ts = AdaBoost_clf.predict(X_test)
svc_pred_ts = svc_clf.predict(X_test)

In [65]:
input_vec = []

In [66]:
for i in range(len(rf_pred_ts)):
    tmp1=[]
    tmp1.append(convert_bool(rf_pred_ts[i]))
    tmp1.append(convert_bool(ada_pred_ts[i]))
    tmp1.append(convert_bool(svc_pred_ts[i]))
    input_vec.append(tmp1+list(X_test[i]))
    
y_test = y_test.apply(convert_bool)

In [67]:
# evaluate the model
scores = model.evaluate(np.array(input_vec), y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

22976/23516 [============================>.] - ETA: 0s
acc: 81.08%
